In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, mapping, LineString
from libpysal import weights
import re

In [52]:
# Map path
map_path = r'*\Desktop\Iran Map\gadm41_IRN_1.json' # Please enter your map path

# Loading polygons data
map = gpd.GeoDataFrame.from_file(map_path)

map.head()

,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
0,IRN.1_1,IRN,Iran,Alborz,NA,استانالبرز,Ostan,Province,NA,IR.AL,IR-30,"MULTIPOLYGON (((51.11610 35.73300, 51.19040 35..."
1,IRN.2_1,IRN,Iran,Ardebil,Ardabil,NA,Ostan,Province,NA,IR.AR,NA,"MULTIPOLYGON (((48.50530 38.02510, 48.48990 38..."
2,IRN.3_1,IRN,Iran,Bushehr,BanadervaJazayer-eKhalij-eFa,NA,Ostan,Province,NA,IR.BS,IR-18,"MULTIPOLYGON (((52.67040 27.46070, 52.67380 27..."
3,IRN.4_1,IRN,Iran,ChaharMahallandBakhtiari,Bakhtiari|ChaharmahalvaBakhtiy,NA,Ostan,Province,NA,IR.CM,NA,"MULTIPOLYGON (((51.14740 32.18390, 51.18350 32..."
4,IRN.5_1,IRN,Iran,EastAzarbaijan,Azarbayjan-eKhavari|Azarbaijan-,NA,Ostan,Province,NA,IR.EA,NA,"MULTIPOLYGON (((46.38160 36.97450, 46.37060 36..."


# First Way 

In [53]:
# data path
df_path = r"*\Desktop\Customers data.xlsx"     # Please enter your file path

# Loading data
df = pd.read_excel(df_path)

In [ ]:
# Preprocessing data 
# Create Functions to clean latitude and longitude
def Clean_Lat(Lat):
    Lat = str(Lat).strip()
    Lat = re.findall(r"\d+\.\d+", Lat)
    if len(Lat) > 0:
        Lat = Lat[0]
        if float(Lat) < -90 or float(Lat) > 90:
            Lat = ""
    else:
        Lat = ""
    return Lat

def Clean_Long(Long):
    Long = str(Long).strip()
    Long = re.findall(r"\d+\.\d+", Long)
    if len(Long) > 0:
        Long = Long[0]
        if float(Long) < -180 or float(Long) > 180:
            Long = "" 
    else:
        Long = ""
    return Long

# Applying the function
df['Lat'] = df['Lat'].apply(Clean_Lat)
df['Long'] = df['Long'].apply(Clean_Long)

In [56]:
# Preprocessing Data
# Converting latitude and longitude to Point
geometric_points = []
for xy in zip(df["Long"], df["Lat"]):
    if any(element is None for element in xy):
        geometric_points.append(None)
    else:
        geometric_points.append(Point(xy))

# Define GeoDataframe from preloaded dataframe
geo_locations = gpd.GeoDataFrame(df,  crs = 'EPSG:4326', geometry = geometric_points)

In [83]:
# Join the map and the geodataframe
join = gpd.sjoin(map, geo_locations, predicate = "contains", how = 'left')

# Make groupby to find the customers count in each polygon
join = join[["NAME_1", "Customer Id"]].groupby(["NAME_1", join["geometry"].to_wkt()]).count().reset_index

# rename columns
join.columns = ['Polygon Name', 'geometry', 'Customers Count']

# create a new geodataframe
join['geometry'] = gpd.GeoSeries.from_wkt(join['geometry'])
join = gpd.GeoDataFrame(join)

In [98]:
# Create a map to visualize the data of each polygon by color
# Define your bins and labels
bins = [0, 1, 200, 300, 400, 500]
labels = ['Label 1: 0', 'Label 2: 1-199', 'Label 3: 200-299', 'Label 4: 300-399', 'Label 5: 400-499']

# Use pd.cut to assign labels based on the bins
join["label"] = pd.cut(join["Customers Count"], bins=bins, labels=labels, include_lowest=True, right=False)

# Creating a Folium Map object
m = folium.Map(location=[33.56287808258443, 53.75348137856239], zoom_start=5, tiles='OpenStreetMap', crs='EPSG3857', width='70%', height='70%', position='left')

# Plotting the result 
join.explore(
    m=m,
    column='label',
    width='70%',
    height='70%',
    legend=True,
    style_kwds={'fillOpacity': 0.5, 'opacity': 0.0, 'color': 'black', 'weight': 1},
    cmap='YlGn_r'
)

# Adding custom legend
legend_html = """
    <div style="margin: 10px; background-color: white; border-radius: 5px; padding: 10px; border: 2px solid grey; font-size: 14px;">
        <p><strong>Legend</strong></p>
        <p><span style="color: blue;">■</span> Label 1: 1-9</p>
        <p><span style="color: green;">■</span> Label 2: 10-19</p>
        <p><span style="color: orange;">■</span> Label 3: 20-29</p>
        <p><span style="color: red;">■</span> Label 4: 30-39</p>
    </div>
"""

# Display the map
m


In [104]:
# Show center of each polygon with pins

# Create center column for the centroid of each district
join['center'] = join.centroid


# Build markers and popups
for row in join.iterrows():
    row_values = row[1]
    center_point = row_values['center']
    location = [center_point.y, center_point.x]
    popup = ('District Name: ' + str(row_values['Polygon Name']) + 
             ';  ' + 'Total Cutomers: ' + str(row_values['Customers Count']))
    marker = folium.Marker(location = location, popup = popup)
    marker.add_to(m)
m

# Second Way
using folium.Choropleth

In [202]:
# data path
df_path = r"*\Desktop\Customers data.xlsx"    # Please enter your file path

# Loading data
df = pd.read_excel(df_path)

In [ ]:
# Preprocessing data 
# Create Functions to clean latitude and longitude
def Clean_Lat(Lat):
    Lat = str(Lat).strip()
    Lat = re.findall(r"\d+\.\d+", Lat)
    if len(Lat) > 0:
        Lat = Lat[0]
        if float(Lat) < -90 or float(Lat) > 90:
            Lat = ""
    else:
        Lat = ""
    return Lat

def Clean_Long(Long):
    Long = str(Long).strip()
    Long = re.findall(r"\d+\.\d+", Long)
    if len(Long) > 0:
        Long = Long[0]
        if float(Long) < -180 or float(Long) > 180:
            Long = "" 
    else:
        Long = ""
    return Long

# Applying the function
df['Lat'] = df['Lat'].apply(Clean_Lat)
df['Long'] = df['Long'].apply(Clean_Long)

In [203]:
# Preprocessing Data
# Converting latitude and longitude to Point
geometric_points = []
for xy in zip(df["Long"], df["Lat"]):
    if any(element is None for element in xy):
        geometric_points.append(None)
    else:
        geometric_points.append(Point(xy))

# Define GeoDataframe from preloaded dataframe
geo_locations = gpd.GeoDataFrame(df,  crs = 'EPSG:4326', geometry = geometric_points)

In [204]:
# Join the map and the geodataframe
join = gpd.sjoin(map, geo_locations, predicate = "contains", how = 'left')

# Make groupby to find the customers count in each polygon
join = join[["NAME_1", "Customer Id"]].groupby(["NAME_1", join["geometry"].to_wkt()]).count().reset_index

# rename columns
join.columns = ['Polygon Name', 'geometry', 'Customers Count']

# create a new geodataframe
join['geometry'] = gpd.GeoSeries.from_wkt(join['geometry'])
join = gpd.GeoDataFrame(join)

In [207]:
# Define the areas of each polygon
join["area"] = join.area
# Define the density of each polygon
join["density"] = join.apply(lambda row: row["Customers Count"]/row["area"], axis = 1)
join.crs = "epsg:4326"

In [208]:
# Creating a Folium Map object
m = folium.Map(location=[33.56287808258443, 53.75348137856239], zoom_start=5, tiles='OpenStreetMap', crs='EPSG3857', width='70%', height='70%', position='left')


folium.Choropleth(geo_data = join, 
                 name = 'geometry',
                 data = join, 
                 columns = ["Customers Count", "density"],
                 fill_color = "YlGn", 
                 key_on='feature.properties.Customers Count').add_to(m)
folium.LayerControl().add_to(m)

# Create center column for the centroid of each district
join['center'] = join.centroid.to_crs("4326")

# Build markers and popups
for row in join.iterrows():
    row_values = row[1]
    center_point = row_values['center']
    location = [center_point.y, center_point.x]
    popup = ('District Name: ' + str(row_values['Polygon Name']) + 
             ';  ' + 'Total Customers: ' + str(row_values['Customers Count']))
    marker = folium.Marker(location = location, popup = popup)
    marker.add_to(m)
display(m)